In [1]:
#pip install librosa
#!pip install tensorflow-gpu
#import librosa
import tensorflow
import IPython
import librosa

import numpy as np
#import scipy.io.wavfile as wave
import tensorflow as tf
import sys



/home/voiceclone/miniconda3/envs/envTF112/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/voiceclone/miniconda3/envs/envTF112/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/voiceclone/miniconda3/envs/envTF112/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/voiceclone/minicond

In [3]:
tf.reset_default_graph()
sys.path.append('./OpenSeq2Seq')
from open_seq2seq.utils.utils import deco_print, get_base_config, check_logdir,\
                                     create_logdir, create_model, get_interactive_infer_results


# Define the command line arguments that one would pass to run.py here
args_S2T = ["--config_file=OpenSeq2Seq/example_configs/speech2text/ds2_large_8gpus_mp.py",
        "--mode=interactive_infer",
        "--logdir=OpenSeq2Seq/ds2_large/",    
        "--batch_size_per_gpu=1",
        "--num_gpus=1"
]#"--batch_size_per_gpu=32",


# A simpler version of what run.py does. It returns the created model and its saved checkpoint
def get_model(args, scope):
    with tf.variable_scope(scope):
        args, base_config, base_model, config_module = get_base_config(args)
        checkpoint = check_logdir(args, base_config)
        model = create_model(args, base_config, config_module, base_model, None)
    return model, checkpoint

model_S2T, checkpoint_S2T = get_model(args_S2T, "S2T")


# Create the session and load the checkpoints
sess_config = tf.ConfigProto(allow_soft_placement=True)
sess_config.gpu_options.allow_growth = True
sess = tf.InteractiveSession(config=sess_config)
vars_S2T = {}
# vars_T2S = {}
for v in tf.get_collection(tf.GraphKeys.VARIABLES):
    if "S2T" in v.name:
        vars_S2T["/".join(v.op.name.split("/")[1:])] = v

saver_S2T = tf.train.Saver(vars_S2T)

saver_S2T.restore(sess, checkpoint_S2T)


*** Restoring from the latest checkpoint
*** Inference config:
{'batch_size_per_gpu': 1,
 'data_layer': <class 'open_seq2seq.data.speech2text.speech2text.Speech2TextDataLayer'>,
 'data_layer_params': {'dataset_files': [],
                       'input_type': 'spectrogram',
                       'num_audio_features': 160,
                       'shuffle': False,
                       'vocab_file': 'OpenSeq2Seq/open_seq2seq/test_utils/toy_speech_data/vocab.txt'},
 'decoder': <class 'open_seq2seq.decoders.fc_decoders.FullyConnectedCTCDecoder'>,
 'decoder_params': {'alpha': 2.0,
                    'alphabet_config_path': 'open_seq2seq/test_utils/toy_speech_data/vocab.txt',
                    'beam_width': 512,
                    'beta': 1.0,
                    'decoder_library_path': 'ctc_decoder_with_lm/libctc_decoder_with_kenlm.so',
                    'lm_path': 'language_model/4-gram.binary',
                    'trie_path': 'language_model/trie.binary',
                    'use_

/home/voiceclone/miniconda3/envs/envTF112/lib/python3.6/site-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [4]:
# Define the inference function

def infer(wave1_name,wave2_name):#
    
    wav1,sr1 = librosa.load(wave1_name)
    wav2,sr2 = librosa.load(wave2_name)
    print('first people')
    audio1 = IPython.display.Audio(wav1, rate = sr1)
    IPython.display.display(audio1)
    print('second people')
    audio2 = IPython.display.Audio(wav2, rate = sr2)
    IPython.display.display(audio2)
    
    wav1 = librosa.core.resample(wav1, sr1, 16000)
    wav2 = librosa.core.resample(wav2, sr2, 16000)

    

    # Recognize speech
    model_in1 = wav1
    model_in2 = wav2
    
    recognized1 = get_interactive_infer_results(model_S2T, sess, model_in=[model_in1])[0][0]
    recognized2 = get_interactive_infer_results(model_S2T, sess, model_in=[model_in2])[0][0]
    print("Recognized Speech")
    print(recognized1)
    print(recognized2)
    return recognized1, recognized2

In [60]:
# wav1_name = './TEST/DR1/FAKS0/SA1.WAV'
# wav2_name = './TEST/DR1/FAKS0/SA2.WAV'
# while True:
# file_dir = './TEST/DR1/FAKS0/'
#     print('input first audio')
#     wav1_name = input()
#     print('input second audio')
#     wav2_name = input()
#     if wav1_name == "" or wav2_name == "":
#         break

#     wav1_name = file_dir + wav1_name


#     wav2_name = file_dir + wav2_name

# IPython.display.clear_output()
# recognized1, recognized2 = infer(wav1_name,wav2_name)

In [5]:
sys.path.append('./Real-Time-Voice-Cloning')
from encoder.params_model import model_embedding_size as speaker_embedding_size
from utils.argutils import print_args
from synthesizer.inference import Synthesizer
from encoder import inference as encoder
from vocoder import inference as vocoder
from pathlib import Path
import numpy as np
import librosa
import argparse
import torch
import IPython
import os
import sys
import re
import pandas as pd 


In [6]:
def load_model():
    #load arguments
#     tts_parser = argparse.ArgumentParser()#formatter_class = argparse.ArgumentDefaultsHelpFormatter
#     tts_parser.add_argument("--encoder",type = Path, default = 'Real-Time-Voice-Cloning/encoder/saved_models/pretrained.pt')
#     tts_parser.add_argument("--synthesizer",type = Path, default = "Real-Time-Voice-Cloning/synthesizer/saved_models/logs-pretrained/taco_pretrained/")
#     tts_parser.add_argument("--vocoder",type = Path, default = "Real-Time-Voice-Cloning/vocoder/saved_models/pretrained/pretrained.pt")
    
#     args = tts_parser.parse_args()
    
    #cuda device
    device_id = torch.cuda.current_device()
    gpu_properties = torch.cuda.get_device_properties(device_id)
    print(gpu_properties)

    #load model
    #load encoder
    encoder.load_model(Path('Real-Time-Voice-Cloning/encoder/saved_models/pretrained.pt'))
    #load synthesizer
    synthesizer = Synthesizer(Path('Real-Time-Voice-Cloning/synthesizer/saved_models/logs-pretrained/taco_pretrained/'),low_mem = False)
    #vocoder
    vocoder.load_model(Path('Real-Time-Voice-Cloning/vocoder/saved_models/pretrained/pretrained.pt'))
    print('load model successfully')
    return encoder,synthesizer,vocoder

In [7]:
def load_audio(file_dir,voice1,voice2):
    in_fpath1 = Path((file_dir+voice1).replace("\"", "").replace("\'", ""))
    original_wave1, sampling_rate1 = librosa.load(in_fpath1)
    in_fpath2 = Path((file_dir+voice2).replace("\"", "").replace("\'", ""))
    original_wave2, sampling_rate2 = librosa.load(in_fpath2)
    IPython.display.display(IPython.display.Audio(original_wave1, rate = sampling_rate1))
    IPython.display.display(IPython.display.Audio(original_wave2, rate = sampling_rate2))
    return [original_wave1,sampling_rate1,original_wave2,sampling_rate2]

In [8]:
encoder,synthesizer,vocoder = load_model()

_CudaDeviceProperties(name='Tesla P100-PCIE-16GB', major=6, minor=0, total_memory=16280MB, multi_processor_count=56)
Loaded encoder "pretrained.pt" trained to step 1564501
Found synthesizer "pretrained" trained to step 278000
Building Wave-RNN
Trainable Parameters: 4.481M
Loading model weights at Real-Time-Voice-Cloning/vocoder/saved_models/pretrained/pretrained.pt
load model successfully


In [70]:
print('input the file direction')
file_dir = input()
print('input first audio')
wav1_name = input()
print('input second audio')
wav2_name = input()



input the file direction
./TEST/
input first audio
DR8/MSLB0/SX383.WAV
input second audio
DR8/FMLD0/SX295.WAV


In [71]:
original_wave1,sampling_rate1,original_wave2,sampling_rate2 = load_audio(file_dir,wav1_name,wav2_name)

In [72]:
preprocessed_wave1 = encoder.preprocess_wav(original_wave1,sampling_rate1)
preprocessed_wave2 = encoder.preprocess_wav(original_wave2, sampling_rate2)
print('preprocess the waveform')
#create embedding
embeds = [encoder.embed_utterance(preprocessed_wave1),encoder.embed_utterance(preprocessed_wave2)]
print('create the speaker embedding')
#synthesize the text

recognized1, recognized2 = infer(file_dir + wav1_name,file_dir + wav2_name)
texts = [recognized2,recognized1]

#
specs = synthesizer.synthesize_spectrograms(texts,embeds)


preprocess the waveform
create the speaker embedding
first people


second people


Recognized Speech
the carpet cleaners sham poute our oriental rug
if caroll comes to morrow haveer arrange for a meeting it too


In [73]:
wrap1 = specs[0]
wrap2 = specs[1]
print('synthesize the text')

#vocoder
generated_wave1 = vocoder.infer_waveform(wrap1)
generated_wave2 = vocoder.infer_waveform(wrap2)

synthesize the text
{| ████████████████ 66500/67200 | Batch Size: 7 | Gen Rate: 6.1kHz | }

In [74]:
IPython.display.display(IPython.display.Audio(generated_wave1, rate = synthesizer.sample_rate))
IPython.display.display(IPython.display.Audio(generated_wave2, rate = synthesizer.sample_rate))


In [75]:
speaker1_name = (file_dir + wav1_name).split('/')[-2]
speaker2_name = (file_dir + wav2_name).split('/')[-2]

In [76]:
os.mkdir('./swapping_result/%s&%s'%(speaker1_name,speaker2_name))


In [77]:
store_path = Path('./swapping_result/%s&%s/%s_swapped.WAV' % (speaker1_name,speaker2_name,speaker1_name))
librosa.output.write_wav(store_path, generated_wave1.astype(np.float32), synthesizer.sample_rate)
store_path = Path('./swapping_result/%s&%s/%s_swapped.WAV' % (speaker1_name,speaker2_name,speaker2_name))
librosa.output.write_wav(store_path, generated_wave2.astype(np.float32), synthesizer.sample_rate)

In [78]:
store_path = Path('./swapping_result/%s&%s/%s_original.WAV' % (speaker1_name,speaker2_name,speaker1_name))
librosa.output.write_wav(store_path, original_wave1.astype(np.float32), sampling_rate1)
store_path = Path('./swapping_result/%s&%s/%s_original.WAV' % (speaker1_name,speaker2_name,speaker2_name))
librosa.output.write_wav(store_path, original_wave2.astype(np.float32), sampling_rate2)